# Predicting NFL Game Outcomes
## Rohan Lewis, Michael Nercessian, Matthew Armstrong

# 2019 NFL Game by Game Data

We first manually downloaded our data from this source: https://www.pro-football-reference.com/teams/

Our first step will be to import all of the modules we need, and then load the data.

In [81]:
import Pkg
Pkg.add("CSV")
Pkg.add("Plots")
Pkg.add("DataFrames")
Pkg.add("Statistics")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [82]:
using Random
using CSV
using Plots
using DataFrames
using Statistics

In [83]:
bills_19 = CSV.read("Datasets/2019 Teams/bills_2019.csv")
nfl_teams = CSV.read("Datasets/nfl_teams.csv")
showall(nfl_teams)

32×5 DataFrame
│ Row │ ID    │ Name                 │ Abbreviation │ Conference │ Division │
│     │ Int64 │ String               │ String       │ String     │ String   │
├─────┼───────┼──────────────────────┼──────────────┼────────────┼──────────┤
│ 1   │ 1     │ Arizona Cardinals    │ ARI          │ NFC        │ West     │
│ 2   │ 2     │ Atlanta Falcons      │ ATL          │ NFC        │ South    │
│ 3   │ 3     │ Baltimore Ravens     │ BAL          │ AFC        │ North    │
│ 4   │ 4     │ Buffalo Bills        │ BUF          │ AFC        │ East     │
│ 5   │ 5     │ Carolina Panthers    │ CAR          │ NFC        │ South    │
│ 6   │ 6     │ Chicago Bears        │ CHI          │ NFC        │ North    │
│ 7   │ 7     │ Cincinnati Bengals   │ CIN          │ AFC        │ North    │
│ 8   │ 8     │ Cleveland Browns     │ CLE          │ AFC        │ North    │
│ 9   │ 9     │ Dallas Cowboys       │ DAL          │ NFC        │ East     │
│ 10  │ 10    │ Denver Broncos       │ DEN       

In [84]:
abbreviations = [nfl_teams[:, 1] nfl_teams[:, 3]]

32×2 Array{Any,2}:
  1  "ARI"
  2  "ATL"
  3  "BAL"
  4  "BUF"
  5  "CAR"
  6  "CHI"
  7  "CIN"
  8  "CLE"
  9  "DAL"
 10  "DEN"
 11  "DET"
 12  "GB"
 13  "HOU"
  ⋮  
 21  "NYG"
 22  "NYJ"
 23  "LV"
 24  "PHI"
 25  "PIT"
 26  "LAC"
 27  "SF"
 28  "SEA"
 29  "LAR"
 30  "TB"
 31  "TEN"
 32  "WAS"

In [85]:
team_names = nfl_teams[:, 2]
team_names_split = String.(reduce(hcat,split.(team_names,' ')))
team_names_split = team_names_split[2, :]
team_names_split = [nfl_teams[:, 1] map(lowercase, team_names_split)]
team_names_split[19:32, 2]

14-element Array{Any,1}:
 "patriots"
 "saints"
 "giants"
 "jets"
 "raiders"
 "eagles"
 "steelers"
 "chargers"
 "49ers"
 "seahawks"
 "rams"
 "buccaneers"
 "titans"
 "redskins"

In [88]:
team_paths = []
for i in 1:size(team_names_split,1)
    team_names_split[i,2] = "Datasets/2019 Teams/" * team_names_split[i,2] * "_2019.csv"
end
team_names_split

32×2 Array{Any,2}:
  1  "Datasets/2019 Teams/Datasets/2019 Teams/cardinals_2019.csv_2019.csv"
  2  "Datasets/2019 Teams/Datasets/2019 Teams/falcons_2019.csv_2019.csv"
  3  "Datasets/2019 Teams/Datasets/2019 Teams/ravens_2019.csv_2019.csv"
  4  "Datasets/2019 Teams/Datasets/2019 Teams/bills_2019.csv_2019.csv"
  5  "Datasets/2019 Teams/Datasets/2019 Teams/panthers_2019.csv_2019.csv"
  6  "Datasets/2019 Teams/Datasets/2019 Teams/bears_2019.csv_2019.csv"
  7  "Datasets/2019 Teams/Datasets/2019 Teams/bengals_2019.csv_2019.csv"
  8  "Datasets/2019 Teams/Datasets/2019 Teams/browns_2019.csv_2019.csv"
  9  "Datasets/2019 Teams/Datasets/2019 Teams/cowboys_2019.csv_2019.csv"
 10  "Datasets/2019 Teams/Datasets/2019 Teams/broncos_2019.csv_2019.csv"
 11  "Datasets/2019 Teams/Datasets/2019 Teams/lions_2019.csv_2019.csv"
 12  "Datasets/2019 Teams/Datasets/2019 Teams/packers_2019.csv_2019.csv"
 13  "Datasets/2019 Teams/Datasets/2019 Teams/texans_2019.csv_2019.csv"
  ⋮  
 21  "Datasets/2019 Teams/Datase

In [87]:
# now let's create some DataFrame variables
cardinals_19 = CSV.read(team_names_split[1,2])
falcons_19 = CSV.read(team_names_split[2,2])
ravens_19 = CSV.read(team_names_split[3,2])
bills_19 = CSV.read(team_names_split[4,2])
panthers_19 = CSV.read(team_names_split[5,2])
bears_19 = CSV.read(team_names_split[6,2])
bengals_19 = CSV.read(team_names_split[7,2])
browns_19 = CSV.read(team_names_split[8,2])
cowboys_19 = CSV.read(team_names_split[9,2])
broncos_19 = CSV.read(team_names_split[10,2])
lions_19 = CSV.read(team_names_split[11,2])
packers_19 = CSV.read(team_names_split[12,2])
texans_19 = CSV.read(team_names_split[13,2])
colts_19 = CSV.read(team_names_split[14,2])
jaguars_19 = CSV.read(team_names_split[15,2])
chiefs_19 = CSV.read(team_names_split[16,2])
dolphins_19 = CSV.read(team_names_split[17,2])
vikings_19 = CSV.read(team_names_split[18,2])
patriots_19 = CSV.read(team_names_split[19,2])
saints_19 = CSV.read(team_names_split[20,2])
giants_19 = CSV.read(team_names_split[21,2])
jets_19 = CSV.read(team_names_split[22,2])
raiders_19 = CSV.read(team_names_split[23,2])
eagles_19 = CSV.read(team_names_split[24,2])
steelers_19 = CSV.read(team_names_split[25,2])
chargers_19 = CSV.read(team_names_split[26,2])
# note: name different than csv file name
fortyniners_19 = CSV.read(team_names_split[27,2])
seahawsks_19 = CSV.read(team_names_split[28,2])
rams_19 = CSV.read(team_names_split[29,2])
buccaneers_19 = CSV.read(team_names_split[30,2])
titans_19 = CSV.read(team_names_split[31,2])
redskins_19 = CSV.read(team_names_split[32,2])

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
,String,String?,String?,String?,String?,String?,String?,String?
1,Week,Day,Date,missing,missing,missing,OT,Rec
2,1,Sun,September 8,1:00PM ET,boxscore,L,missing,0-1
3,2,Sun,September 15,1:00PM ET,boxscore,L,missing,0-2
4,3,Mon,September 23,8:15PM ET,boxscore,L,missing,0-3
5,4,Sun,September 29,1:00PM ET,boxscore,L,missing,0-4
6,5,Sun,October 6,1:00PM ET,boxscore,L,missing,0-5
7,6,Sun,October 13,1:00PM ET,boxscore,W,missing,1-5
8,7,Sun,October 20,1:00PM ET,boxscore,L,missing,1-6
9,8,Thu,October 24,8:20PM ET,boxscore,L,missing,1-7
